In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import time
import tensorflow as tf
import p8_util
import p8_util_config

In [5]:
import p8_util
datadir = './data'

filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, feature_shape = p8_util.load_dataset(filename_dataset,is_label_encoded=True)

print("\nDataset : {}".format((x_train.shape, x_test.shape, y_train.shape, y_test.shape, nClasses, feature_shape)))

p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
(414, 224, 224, 3) (47, 224, 224, 3) (414, 3) (47, 3)

Dataset : ((414, 224, 224, 3), (47, 224, 224, 3), (414,), (47,), 3, (224, 224, 3))


In [6]:
OUTPUT_DIR = './tmp/baseline'
OUTPUT_DIR_TB = './tmp'
is_tensorboard = True

In [7]:
# this will kill the processes for Tensorboard
#is_tensorboard = True
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

kill: (23859)kill: (23865): Aucun processus de ce type


#### Increase of train dataset against test dataset.

In [8]:
OUTPUT_DIR

'./tmp/baseline'

In [9]:
import p8_util
print("Input output dir= {}".format(OUTPUT_DIR))
oNNAdaNetBuilder = p8_util.create_nn_builder(feature_shape,OUTPUT_DIR)
oNNAdaNetBuilder.show()

Input output dir= ./tmp/baseline


Adanet outputdir     : ............................ ./tmp/baseline/DNN
Adanet output log    : ............................ ./tmp/baseline/DNN/DNN
Adanet boosting iter.: ............................ 10
Adanet global steps  : ............................ 200
Adanet iter per boost: ............................ 20
NN type              : ............................ DNN
Features shape       : ............................ (224, 224, 3)
Number of layers     : ............................ 3
Dropout rate         : ............................ 0.0
Seed value           : ............................ 42
Nb of classes (logit): ............................ 3
Adanet regularization: ............................ 1e-05
Weights initializer  : ............................ xavier
Batch normalization  : ............................ True
Learn mixture weights: ............................ True


Number of layers     : ............................ 3
Hidden units         : ..

In [10]:
p8_util_config.NUM_EPOCHS, p8_util_config.BATCH_SIZE, p8_util_config.NUM_EPOCHS * p8_util_config.BATCH_SIZE, 
224//34

6

In [11]:
p8_util_config.dict_adanet_config

{'adanet_feature_columns': None,
 'adanet_feature_shape': None,
 'adanet_tf_head': None,
 'adanet_lambda': 1e-05,
 'adanet_is_learn_mixture_weights': True,
 'adanet_initial_num_layers': 0,
 'adanet_num_layers': None,
 'adanet_nn_candidate': 2,
 'adanet_output_dir': './tmp/adanet',
 'adanet_nn_layer_config': {'nn_type': 'DNN',
  'nn_dropout_rate': 0.0,
  'nn_batch_norm': True,
  'nn_logit_dimension': 3,
  'nn_optimizer': <tensorflow.python.training.rmsprop.RMSPropOptimizer at 0x7f23d3f80470>,
  'nn_seed': 42,
  'nn_initializer_name': 'xavier',
  'nn_layer_config': {'dnn_layer_num': 3, 'dnn_hidden_units': 128}}}

In [12]:
import p8_util

batch_test = 20
input_fn_param={'num_epochs':p8_util_config.NUM_EPOCHS,\
                'batch_size':p8_util_config.BATCH_SIZE,\
                'feature_shape': oNNAdaNetBuilder.feature_shape,\
               }
train_spec=tf.estimator.TrainSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train,input_fn_param),\
        max_steps=p8_util_config.TRAIN_STEPS)

eval_spec=tf.estimator.EvalSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train,input_fn_param),\
        steps=None,\
        throttle_secs=1)

In [13]:
params = {'net_builder': oNNAdaNetBuilder}
    
classifier = tf.estimator.Estimator(model_fn=p8_util.custom_model_fn, params=params, config=oNNAdaNetBuilder.classifier_config)

INFO:tensorflow:Using config: {'_model_dir': './tmp/baseline/DNN/DNN', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f23d3f989b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0515 22:54:44.142863 139793697040192 estimator.py:201] Using config: {'_model_dir': './tmp/baseline/DNN/DNN', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f23d3f989b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:
import p8_util

start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(classifier, train_spec=train_spec, eval_spec=eval_spec)
end_time = time.time()

INFO:tensorflow:Not using Distribute Coordinator.


I0515 22:54:44.198596 139793697040192 estimator_training.py:185] Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


I0515 22:54:44.199835 139793697040192 training.py:610] Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 10 or save_checkpoints_secs None.


...

I0515 23:04:19.514893 139793697040192 estimator.py:1979] Saving dict for global step 198: DNN_Eval_accuracy = 0.33816424, global_step = 198, loss = 1.0984128


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 198: ./tmp/baseline/DNN/DNN/model.ckpt-198


I0515 23:04:19.516018 139793697040192 estimator.py:2039] Saving 'checkpoint_path' summary for global step 198: ./tmp/baseline/DNN/DNN/model.ckpt-198


INFO:tensorflow:Loss for final step: 1.10066.


I0515 23:04:19.536989 139793697040192 estimator.py:359] Loss for final step: 1.10066.


In [15]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))
print("------------------------------------------------\n")



------------------------------------------------
Time (sec) 575.3410842418671
DNN_EVAL_ACCURACY: 0.33816424012184143
LOSS: 1.0984127521514893
GLOBAL_STEP: 198
------------------------------------------------



In [16]:
oNNAdaNetBuilder.output_dir_log

'./tmp/baseline/DNN/DNN'

In [17]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6007 &'
        .format(oNNAdaNetBuilder.output_dir_log)
    )


    get_ipython().system_raw('./assets/ngrok http 6007 &')

In [18]:
500//10

50

In [19]:
138*3
epochs=20
mini_batch_size=138
mini_batch_number =414//mini_batch_size
train_step = 200
epochs = train_step//mini_batch_number
train_step,epochs

(200, 66)